In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6_leguvi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6_leguvi
  Resolved https://github.com/huggingface/transformers.git to commit 41a2f3529c6b56866c317031375ffd3e7b8bea01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6862992 sha256=bcd0cbc321dedf70b6784605c648f2f0f75ba01ff4ed8e67ea8e05589806106a
  Stored in directory: /tmp/pip-ephem-wheel-cache-04ouhmha/whee

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca")
model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca").to('cuda:0')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
token = tokenizer('안녕하세요', return_tensors='pt').to('cuda:0')
model.generate(input_ids=token['input_ids'], attention_mask=token['attention_mask'])

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    2, 29871, 31734,   238,   136,   152, 30944, 31578, 31527, 29892,
         29871,   239,   163,   131, 31081, 29871,   239,   192,   151, 31493]],
       device='cuda:0')

In [4]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [5]:
def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    
    return gen_text.replace(x, '')

In [6]:
prompt = "고양이를 잘 기르는 법"
generated_text = gen(prompt)
print("==================================")
print("Response: ")
print(generated_text)
print("==================================")

Response: 
고양이의 욕심이 높은 법입니다! 먼저, 귀를 잡아 주세요. 그런 다음, 반응이 좋은 것을 추가하는 것을하면 좋습니다. 그리고 반으로 손가락을 움
